In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# hyperparameters
block_size = 128
batch_size = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 50
n_embd = 128
n_head = 4
n_layer = 4
learning_rate = 1e-3

text = open('../data/shakespeare.txt', encoding='utf-8').read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

# create mapping
itoc = { i: c for i, c in enumerate(chars)}
ctoi = { c: i for i, c in enumerate(chars)}
encode = lambda s: [ctoi[c] for c in s]
decode = lambda l: ''.join([itoc[i] for i in l])

data = torch.tensor(encode(text))
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            logits, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        
        B,T,C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        # compute attention scores
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        #value
        v = self.value(x)
        out = wei @ v
        
        return out
    
class MultiHeadAttention(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        
    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        return x
    
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd)
        )
        
    def forward(self, x):
        x = self.net(x)
        return x
    
class Block(nn.Module):
    def __init__(self):
        super().__init__()
        
        head_size = n_embd // n_head
        
        self.sa = MultiHeadAttention(head_size)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        x = x + self.ln1(self.sa(x))
        x = x + self.ln2(self.ffwd(x))
        return x
    
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
model = Transformer().to(device)
print(f'{sum(p.numel() for p in model.parameters())/1e6}, M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


epochs = 500
for epoch in range(epochs):
    
    if epoch % (epochs // 10) == 0 or epoch == epochs-1:
        out = estimate_loss()
        print(f"epoch : {epoch} - train : {out['train']} - val: {out['val']}")
        
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

0.824897, M parameters
epoch : 0 - train : 4.429075717926025 - val: 4.4235687255859375
epoch : 50 - train : 2.641479253768921 - val: 2.6436307430267334
epoch : 100 - train : 2.531541347503662 - val: 2.541222333908081
epoch : 150 - train : 2.505254030227661 - val: 2.4984993934631348
epoch : 200 - train : 2.4560697078704834 - val: 2.4793291091918945
epoch : 250 - train : 2.390014410018921 - val: 2.4046638011932373
epoch : 300 - train : 2.312854528427124 - val: 2.3417210578918457
epoch : 350 - train : 2.2492501735687256 - val: 2.2827260494232178
epoch : 400 - train : 2.180971622467041 - val: 2.2218854427337646
epoch : 450 - train : 2.1134157180786133 - val: 2.1633567810058594
epoch : 499 - train : 2.053980827331543 - val: 2.109541893005371
